In [1]:
using DataStructures
using FileIO
using Meshes
using MeshIO
using Makie
using GLMakie
using GeometryBasics
using ProgressBars
using Ripserer

In [2]:
include("contractions.jl")
include("visualisation.jl")
include("preprocessing.jl")

loadsc (generic function with 1 method)

In [3]:
bunidata = loadsc("ply/bunny.ply")

# rotate by 90 degrees around the x-axis
println("Get rotated lmao")
for vertex in bunidata.coords
    vertex[2], vertex[3] = vertex[2] * cos(π / 2) - vertex[3] * sin(π / 2), vertex[2] * sin(π / 2) + vertex[3] * cos(π / 2)
end

buni = initialContractedSimplicialComplex2D(bunidata)
butthole = getborder(buni.contracted)
# convert Set{Tuple{Int64, Int64}}) to Array{GeometryBasics.Point{3,Float32}})
butthole_array = collect(butthole)
butthole = [GeometryBasics.Point3(buni.contracted.coords[butthole_array[i][1]]...) for i in 1:length(butthole_array)]
fillholes!(buni.contracted)

0.0%┣                                          ┫ 0/69.5k [00:00<-10:-31, -0s/it]
0.0%┣                                        ┫ 1/69.5k [00:01<Inf:Inf, InfGs/it]
2.6%┣█                                      ┫ 1.8k/69.5k [00:01<00:47, 1.4kit/s]
6.5%┣██▌                                    ┫ 4.5k/69.5k [00:01<00:19, 3.4kit/s]
10.8%┣████                                  ┫ 7.5k/69.5k [00:01<00:11, 5.5kit/s]
14.9%┣█████▌                               ┫ 10.4k/69.5k [00:01<00:08, 7.3kit/s]
18.7%┣███████                              ┫ 13.0k/69.5k [00:01<00:06, 8.8kit/s]
22.7%┣████████▏                           ┫ 15.8k/69.5k [00:02<00:05, 10.4kit/s]
26.5%┣█████████▌                          ┫ 18.4k/69.5k [00:02<00:04, 11.7kit/s]
30.2%┣██████████▉                         ┫ 21.0k/69.5k [00:02<00:04, 12.9kit/s]
34.0%┣████████████▎                       ┫ 23.6k/69.5k [00:02<00:03, 14.2kit/s]
38.0%┣█████████████▊                      ┫ 26.4k/69.5k [00:02<00:03, 15.3kit/s]
42.3%┣███████████████▎      

Get rotated lmao


99.0%┣███████████████████████████████████▋┫ 68.8k/69.5k [00:03<00:00, 27.1kit/s]
100.0%┣███████████████████████████████████┫ 69.5k/69.5k [00:03<00:00, 27.2kit/s]
100.0%┣███████████████████████████████████┫ 69.5k/69.5k [00:03<00:00, 27.2kit/s]


Finding buttholes
Buttholes left: 223
Buttholes left: 184
Buttholes left: 142
Buttholes left: 62
Buttholes left: 22
Constipating 5 buttholes


0.0%┣                                                ┫ 0/5 [00:00<00:00, -0s/it]
20.0%┣████████▋                                  ┫ 1/5 [00:00<Inf:Inf, InfGs/it]
100.0%┣██████████████████████████████████████████████┫ 5/5 [00:00<00:00, 40it/s]
100.0%┣██████████████████████████████████████████████┫ 5/5 [00:00<00:00, 40it/s]


In [4]:
println("Calculating fundamental quadratics")
calculateFundamentalQuadratics!(buni)

println("Constructing initial priority queue")
pq = PriorityQueue()
for (edge, triangle) in ProgressBar(buni.contracted._edge_to_triangles)
    pq[edge] = error(buni, edge)
end

Calculating fundamental quadratics
Constructing initial priority queue


0.0%┣                                           ┫ 0/104.5k [00:00<00:-3, -0s/it]
0.0%┣                                       ┫ 1/104.5k [00:01<Inf:Inf, InfGs/it]
1.4%┣▌                                     ┫ 1.4k/104.5k [00:01<00:45, 2.3kit/s]
5.0%┣██                                    ┫ 5.2k/104.5k [00:01<00:13, 7.7kit/s]
8.7%┣███▏                                 ┫ 9.1k/104.5k [00:01<00:08, 12.5kit/s]
12.2%┣████▎                              ┫ 12.8k/104.5k [00:01<00:06, 16.4kit/s]
15.4%┣█████▍                             ┫ 16.1k/104.5k [00:01<00:05, 19.4kit/s]
18.5%┣██████▌                            ┫ 19.3k/104.5k [00:01<00:04, 21.9kit/s]
21.6%┣███████▌                           ┫ 22.5k/104.5k [00:01<00:03, 24.2kit/s]
24.7%┣████████▋                          ┫ 25.8k/104.5k [00:01<00:03, 26.3kit/s]
27.7%┣█████████▊                         ┫ 28.9k/104.5k [00:01<00:03, 28.0kit/s]
30.5%┣██████████▋                        ┫ 31.8k/104.5k [00:01<00:02, 29.4kit/s]
33.3%┣███████████▋          

In [5]:
i = 0
while true #i < 7486 # problematic edge: (31639, 31761)
    global myedge
    try
        global myedge
        myedge = dequeue!(pq)
        while !issafe(buni.contracted, myedge)
            myedge = dequeue!(pq)
        end
    catch e
        break
    end

    new_vertex = contract!(buni, myedge)
    for edge in buni.contracted._vertex_to_edges[new_vertex]
        pq[edge] = error(buni, edge)
    end
    i += 1
end
println(i)

34835


4-element Vector{Vector{Float32}}:
 [-0.07527522, -0.024266982, 0.14042638]
 [-0.022113241, -0.05638052, 0.05141888]
 [-0.05251966, 0.029483637, 0.045751806]
 [0.060952816, -0.009483511, 0.098919116]

In [ ]:
vertices1 = [buni.original.coords[vertex] for vertex in buni.original.vertices]
rips1 = ripserer(vertices1; dim_max=3)